## Stream

In [ ]:
import pandas as pd
from py.functions import criar_stream

FILE = 'data/UNSW-NB15_1.csv'
df_raw = pd.read_csv(FILE)

# Esta função única faz todo o trabalho de Carga, Limpeza, Ordenação e Criação
stream, le_global, features = criar_stream(
    df=df_raw,
    target_label_col='Label', 
    remove_redundant=False
)

# --- Verificações e Logs ---
print("-" * 40)
print(f"Processamento Concluído.")
print(f"Features Originais no CSV: {len(df_raw.columns)}")
print(f"Features Mantidas: {len(features)}")
print("-" * 40)
print("Schema da Stream (CapyMOA):")
print(stream.get_schema())

## Treinamento e Avaliação

In [ ]:
import time
from capymoa.datasets import ElectricityTiny
from capymoa.anomaly import (
    OnlineIsolationForest,
    HalfSpaceTrees,
    AdaptiveIsolationForest
)
from capymoa.evaluation import AnomalyDetectionEvaluator
from capymoa.evaluation.results import PrequentialResults

# Configuração Inicial
stream = ElectricityTiny()
schema = stream.get_schema()
learner = OnlineIsolationForest(schema=schema) 
evaluator = AnomalyDetectionEvaluator(schema)

# Iniciar contagem de tempo
start_time = time.time()
start_cpu = time.process_time()

# Loop de Processamento 
while stream.has_more_instances():
    instance = stream.next_instance()
    
    # Detecção (Score)
    proba = learner.score_instance(instance)
    
    # Atualização das Métricas
    evaluator.update(instance.y_index, proba)
    
    # Treinamento
    learner.train(instance)

# Parar contagem de tempo
wallclock = time.time() - start_time
cpu_time = time.process_time() - start_cpu

# Criar o objeto de Resultados Finais
results = PrequentialResults(
    learner=str(learner),           # Nome do modelo como string
    stream=stream,                  # O objeto stream original
    wallclock=wallclock,            # Tempo total de execução
    cpu_time=cpu_time,              # Tempo de CPU
    max_instances=2000,             # Número máximo de instâncias processadas
    cumulative_evaluator=evaluator  # O avaliador que acumulou as métricas!
)

# Exibir os resultados
print(f"Modelo: {results.learner}") 
print(f"AUC: {results.auc():.2f}")
print(f"Tempo Total (Wallclock): {results.wallclock():.4f} segundos")
print(f"Tempo de CPU: {results.cpu_time():.4f} segundos")
print(f"metrics_dict: {results.metrics_dict()}")

AUC Final: 0.39
Modelo: Online Isolation Forest
Tempo Total (Wallclock): 14.4538 segundos
Tempo de CPU: 15.5781 segundos
metrics_dict: {'instances': 2000.0, 'auc': 0.39197960565924145, 's_auc': 0.024597105649897547, 'Accuracy': 0.6025, 'Kappa': 0.004836868353061459, 'Pos/Neg ratio': 0.6556291390728477, 'G-Mean': 0.13659130436104627, 'Recall': 0.01893939393939394, 'KappaM': -0.008883248730964334}


## Resultados

In [ ]:
import numpy as np
import matplotlib.pyplot as plt